# ch.6 도커를 활용한 Flask 실행
> 전 시간에 django를 홯용하여 웹서비스를 실행했습니다. 6장에서는 Flask를 황용해 웹서비스를 실행해봅시다. 
> Flask라이브러리를 설치하고 실행을 위해 필요한 네트워크를 설정합니다.


## 1. Flask 연결

### 1. 가상환경 설치
```
conda create -n docker_env python=3.11.6
conda activate docker_env
pip install flask 
```

### 2. 네트워크 설정
브라우저에서 192.168.0.1을 입력하여 공유기의 포트포워딩을 진행합니다. 8001,81번을 등록하세요.<br>
내가 접속한 네트워크를 체크하거나 현재 컴퓨터의 LAN주소를 입력하세요<br>
![](image/포트포워딩.png)

기존에 사용중인 외부포트가 있다면 변경해서 겹치지 않도록 진행하세요<br>
![image.png](image/포트포워딩2.png)

### 3. 파일 작성
> flask를 실행하는 python 코드를 만들어봅시다

```
cd ex01
mkdir wsapp # 만들 폴더이름을 정하세요
cd wsapp 
notepad main.py
```
메모장이 열리면 다음을 복/붙하세요
```
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello_world():
    return "Hello, World!"

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8081)
```
보안이 작동하지만 내부망이기에 잘 접속되는 걸 볼수 있습니다.

![image.png](image/스크린.png)

![image.png](image/성공.png)
<br>
> 단 외부망에서 우리집의 서버에 접속하려면 윈도우의 보안 설정에서 인바운드설정-특정포트 를 오픈해 주어야 접속이 가능합니다.<br>
![image.png](image/보안.png)

## 2. Nginx,Flsek 연동후 실행
> Nginx(웹서버)와 Flask(WAS)를 컨테이너 형태로 연동한 후 실행하겠습니다. 이를 위해 실습디렉토리를 정리하고 Flask와 Nginx 이미지를 각각 빌듷란 후 컨테이너로 실행합니다.

2. 플라스크 디렉초


### 1. 플라스크디렉토리 복사
```
cd ..
mkdir ex02
cp -r ex01 ex02
cd ex02
mv ex01 myFlask_ws # 이동할 경로이름을 정하세요
```

### 2. 도커파일을 이용한 이미지 빌드
> 도커파일을 이용하면 쉘에 하나하나 입력해야 했던 명령어를 한번에 실행합니다. 하나의 폴더에 반드시 Dockerfile만 존재해야 합니다.(확장자 없음)
```
cd .\myFlask_ws\    # 위에서 설정한 경로로 이동합니다.
notepad requirements.txt
```
설치할라이브러리를 복사하여 붙여넣습니다.
```
flask==3.0.0
gunicorn==20.1.0
```
이어서 도커파일도 생성합니다.
```
notepad Dockerfile
```
실행할 명령어를 붙여넣고 저장합니다.
```
FROM python:3.11.6

WORKDIR /usr/src/app

COPY . .

RUN python -m pip install --upgrade pip
RUN pip install -r requirements.txt

WORKDIR ./myapp

CMD gnicorn --bind 0.0.0.0:8001 main:app

EXPOSE 8001

```
노트패드는 확장자가 생기므로 변경 후 도커빌드를 실행하면 해당이미지가 생성됩니다.
```
Rename-Item -Path "Dockerfile.txt" -NewName "Dockerfile"
docker image build . -t  myflask_ws
docker image ls
```
네트워크를 생성하고 컨테이너를 실행합니다.
```
docker network create mynetwork_wsf
docker container run -d --name flasktest --network mynetwork_wsf myflask_ws
docker ps
```

제대로 실행되지 않을때는 로그를 확인하세요
```
docker logs flasktest
```
로그를 확인하여 문제가 있는 부분의 오타 등을 수정한 후 다시 생성할 경우에는
컨테이너 및 이미지를 모두 삭제합니다.
```
docker container rm flasktest
docker image rm myflask_ws
```

### Nginx이미지 빌드
> Nginx도 똑같이 빌드해봅시다
```
cd ..
mkdir myNginx_wsf
cd myNginx_wsf
notepad default.conf
```
config파일에 서버 설정할 내용을 입력합니다. 8081(Flask)로 들어오는 트래픽를 도커내부에서 81 (Nginx)로 넘겨주는 설정입니다.
```
upstream myweb{
    server flasktest:8081;  # 'django'로 수정
}

server {
    listen 81;
    server_name localhost;

    location / {
        proxy_pass http://myweb;
    }
}
```
이번엔 NginX의 도커파일을 만듭니다.
```
notepad Dockerfile
```
다음을 입력하고 저장
```
FROM nginx:1.25.3
RUN rm /etc/nginx/conf.d/default.conf
COPY default.conf /etc/nginx/conf.d/
CMD ["nginx", "-g", "daemon off;"]
```
도커파일 확장자 변경 및 도커 빌드합니다. 
```
Rename-Item -Path "Dockerfile.txt" -NewName "Dockerfile"
docker image build . -t  mynginx_wsf 
docker image ls
docker container rm flasktest
docker image rm myflask_ws
docker container run -d --name nginxtest --network mynetwork_wsf -p81:81 mynginx_wsf
```

## 3. 도커컴포즈를 통한 컨테이너 실행
> 위의 복잡한 과정이 아닌 docker-compose.yml 하나로 두개의 서비스를 만들고 연결하겠습니다.

```
cd 'D:\drive\SelfStudy\ds4th_study\source\도커심화_한권으로_배우는_도커_&_쿠버네티스\ch06'
cp -r ex02\myNginx_wsf ex03ws\myNginx_wsf # 경로 이름 알맞게 변경하세요
cp -r ex02\myFlask_ws ex03ws\myFlask_ws # 경로 이름 알맞게 변경하세요
cd ex03ws
notepad docker-compose.yml
```
저장한 후에는 도커를 실행합니다.
```
docker compose up -d --build
docker ps #실행중인 도커컨테이너를 확인합니다.
docker container logs 521f9d747f8d # 컨테이너ID를  입력하여 log 확인
docker compose down
```
> 도커컴포즈 파일은 각 폴더의 도커파일을 build 실행하고 network를 연결합니다. 이때, depend_on으로 실행순서를 결정할 수 있습니다
![](./image/docker-compose.png)
